In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
import numpy as np


ModuleNotFoundError: No module named 'tensorflow.python.distribute.load_context'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_lfw_people

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X, y = lfw_people.images, lfw_people.target

In [ ]:
X = X / 255.0 # Normalize pixel values to be between 0 and 1



In [ ]:
 X.shape

In [ ]:
X.shape[1] * X.shape[2] #Number of features (pixels)

In [ ]:
len(np.unique(y)) #Number of unique individuals

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i], cmap='gray')
    plt.title(f'Person {y[i]}')
    plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(X[0].ravel(), bins=256, color='gray', alpha=0.7)
plt.title('Pixel Value Distribution in Example Image')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
label_encoder = LabelEncoder()
all_labels = np.concatenate((y_train, y_test))
all_labels_encoded = label_encoder.fit_transform(all_labels)

y_train_encoded = all_labels_encoded[:len(y_train)]
y_test_encoded = all_labels_encoded[len(y_train):]


In [ ]:
label_encoder.classes_

In [ ]:


# Create a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2],1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(5749, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),  
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)


print("X_train shape:", X_train_reshaped.shape)
print("X_test shape:", X_test_reshaped.shape)

In [ ]:
history = model.fit(X_train_reshaped, y_train_encoded, validation_data=(X_test_reshaped, y_test_encoded), epochs=10, batch_size=32)


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)


In [ ]:
test_accuracy

In [ ]:
model_aug = models.Sequential([
   layers.Conv2D(64, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2],1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
model_aug.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
history = model.fit(
    datagen.flow(X_train, y_train_encoded, batch_size=32),
    validation_data=(X_test, y_test_encoded),
    epochs=20
)


In [ ]:
model.save('aug_training_model.h5')
